## A simple math problem

There is a group of 300 people who want to help each other. Unfortunately, a single person can meaningfully keep track of 150 people. Hovewer, all 300 want to cooperate and treat each other fairly. How can they do that?

It's sad that you will not find the problem in any schoolbook. If you did, the initial answer would be simple. Why don't they write down who helped whom and how much?

In [2]:
from dataclasses import dataclass, field
from time import time

@dataclass
class Transaction:
    recipient: str
    sender: str
    amount: float
    timestamp: float = field(default_factory=time)

## Key block

A block of transactions can help people keep track of any help provided. That way you don't need to remember how much help you gave and how much you received.

In [3]:
block = []
block.append(Transaction(
    sender='Alice', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [4]:
block

[Transaction(recipient='Bob', sender='Alice', amount=2.0, timestamp=1543080574.365635),
 Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1543080574.3657246)]

This covers direct person to person cooperation. Everyone is happy but if someone tries to cooperate indirectly, things fall apart. Since anyone can update the block, transactions can be removed. If people try to exchange units of help, there is an incentive to mess with the transaction list.

In order to prevent tampering with transactions in a block we can hash the block and include the hash in the next block. If someone changes transactions, the hash will be different and the tampering will be apparent.

In [5]:
import hashlib

class Block:
    transactions: list
    previous_hash: str = 'coinbase'

    def __init__(self):
        self.transactions = []

    def append(self, t: Transaction):
        self.transactions.append(t)
    
    def __repr__(self):
        return 'transactions: {}\n\nprevious hash: {}'.format(self.transactions.__repr__(), self.previous_hash)
    
    def hash(self):
        return hashlib.sha256(str(self).encode()).hexdigest()
        

In [6]:
block = Block()
block.append(Transaction(
    sender='Alice', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [7]:
block

transactions: [Transaction(recipient='Bob', sender='Alice', amount=2.0, timestamp=1543080684.280628), Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1543080684.2807264)]

previous hash: coinbase

In [8]:
block.hash()

'e1e5db122a41e6feff5813a2a6775e31087d86881ae458c6211ca0bedf0e005a'

Hey, look at that! We got a short and unique (for our purposes) representation of our block and all the transactions in it.

In [9]:
block.append(Transaction(
    sender='Jon', recipient='Bob', amount=1.0
))

In [10]:
block.hash()

'7b485bd5f8cf0971479c38ab4072385bfa12a9612397d7e6f845d1dca5d049f6'

## Verification

Now anyone can put the blocks into a chain and it's easy to prove that no one tampered with prevous transactions.

In [12]:
import itertools


class Chain:
    blocks: list

    def __init__(self):
        """We need initial block"""
        block = Block()
        block.append(Transaction(
            sender='Alice', recipient='Alice', amount=2.0
        ))
        self.blocks = [block]

    def push(self, block: Block):
        block.previous_hash = self.blocks[-1].hash()
        self.blocks.append(block)

    @property
    def tampered(self):
        """
        This gives you the position of an invalid block according to its neighbour to the right.
        If the chain looks good, it returns 0.
        """
        a, b = itertools.tee(self.blocks)
        next(b, None)
        for position, pair in enumerate(zip(a, b)):
            if pair[0].hash() != pair[1].previous_hash:
                return position
        return -1

    @property
    def is_valid(self):
        if self.tampered < 0:
            return True
        return False
        

In [13]:
chain = Chain()

In [14]:
chain.blocks

[transactions: [Transaction(recipient='Alice', sender='Alice', amount=2.0, timestamp=1543080808.9180856)]
 
 previous hash: coinbase]

In [15]:
for _ in range(10):
    block = Block()
    for j in range(3):
        block.append(Transaction(sender='Bob', recipient='Jon', amount=j))
    chain.push(block)

In [16]:
chain.is_valid

True

In [17]:
borrowed = chain.blocks[0].transactions.pop()
borrowed

Transaction(recipient='Alice', sender='Alice', amount=2.0, timestamp=1543080808.9180856)

In [18]:
chain.is_valid

False

In [19]:
chain.tampered

0

Let's put the borrowed transaction back.

In [20]:
chain.blocks[0].transactions.append(borrowed)

In [21]:
chain.is_valid

True

We can only mess with transactions in the last block.

In [22]:
chain.blocks[-1].append(Transaction(recipient='Alice', sender='Jon', amount=5))

In [23]:
chain.is_valid

True

## Equal opportunity to verify

Now, who should have the right to create blocks and add them to the chain? Everyone who wants to! We just need to make sure it's not monopolized by a single person. Somehow we need to give an equal-ish chance for everyone to verify previous blocks and add new ones. Here comes the elegant part. What if we allow the previous hash field to only start with `00`? Our block will have a special `nonce` field that can have anything that makes our block hash start with `00`. If we use a good hashing algorithm, there is no other way but to try different possibilities until we get what we need. This kind of help work should be rewarded. We'll allow that person to assign help units to herself out of nothing.

In [24]:
class Block:
    transactions: list
    previous_hash: str = 'coinbase'
    nonce: str = 'blahblahblah'

    def __init__(self):
        self.transactions = []

    def append(self, t: Transaction):
        self.transactions.append(t)
    
    def __repr__(self):
        return 'transactions: {}\n\nprevious hash: {}\n\nnonce: {}'.format(
            self.transactions.__repr__(), self.previous_hash, self.nonce
        )
    
    def hash(self):
        return hashlib.sha256(str(self).encode()).hexdigest()
        

In [25]:
from uuid import uuid4

def add_block_with_proof_of_work(chain: Chain, block: Block):
    last_block = chain.blocks[-1]
    for i in range(100000):
        last_block.nonce = uuid4()
        temp_hash = last_block.hash()
        if temp_hash.startswith('00'):
            block.previous_hash = temp_hash
            chain.blocks.append(block)
            print('Found hash {} after {} iterations'.format(temp_hash, i))
            break
    else:
        print('After {} iterations, needed hash was not found.'.format(i))    

In [26]:
chain = Chain()

In [27]:
block = Block()
for j in range(3):
    block.append(Transaction(sender='Bob', recipient='Jon', amount=j))

In [28]:
add_block_with_proof_of_work(chain, block)

Found hash 0057e4d1374f4e5668081b19459538f4e37f8a03311eef108ac51678907f7e2d after 587 iterations


In [29]:
chain.blocks

[transactions: [Transaction(recipient='Alice', sender='Alice', amount=2.0, timestamp=1543081199.940345)]
 
 previous hash: coinbase
 
 nonce: 459c51f7-6af2-4dff-9afd-cd2857ac8ac1,
 transactions: [Transaction(recipient='Jon', sender='Bob', amount=0, timestamp=1543081201.4088042), Transaction(recipient='Jon', sender='Bob', amount=1, timestamp=1543081201.4088075), Transaction(recipient='Jon', sender='Bob', amount=2, timestamp=1543081201.4088092)]
 
 previous hash: 0057e4d1374f4e5668081b19459538f4e37f8a03311eef108ac51678907f7e2d
 
 nonce: blahblahblah]

In [30]:
chain.is_valid

True

The facinating part is that we can have any number of arbitrary rules. As long as each helper (a.k.a miner) validates previous work, the system is stable. Each helper gets a reward if her block becomes part of the chain and loses that reward if someone else decides it's wrong. Since it's guaranteed that no one can validate their own work, everyone has to abide by an accepted set of rules.

## Asymmetric encryption to the rescue

There is one last major problem. Anyone can register a transaction from Alice (since she has a lot of help units) to Bob. It will be all valid but Alice might not even know she transferred her help units to Bob.

Sadly, Python doesn't have many encryption features in its standard library. Cryptography is a big field. There are lots of libs that can generate keys like PyCryptodome but we need something simple here. [ecdsa](https://github.com/warner/python-ecdsa) is the simplest to use for signature purposes. After `pip install ecdsa` we can import and use it.

In [31]:
import ecdsa
from base64 import b64decode, b64encode

sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
vk = sk.get_verifying_key()
signature = sk.sign(vk.to_string())
vk.verify(signature, vk.to_string())

True

The idea looks simple but `sk` (Signature Key) and `vk` (Verifying Key) are objects so we can't just put them into a transaction.

In [32]:
sk, vk

(<ecdsa.keys.SigningKey at 0x7fa04822f470>,
 <ecdsa.keys.VerifyingKey at 0x7fa048173940>)

There are helper methods but it still looks ugly.

In [33]:
vk.to_string()

b'\x06\xfa\x84\t3\x9d\x87\x97\xd7\x02\x82\x05d\xcc\x0b\xde1\xc3\xe3|\xab\xf4l\x85\xfe\xce{\x13\xe4\x9d\xe9\xc1\x87\xf5\xf9\xc6\xda\xa4\xa9\x12V\xc3\xe3\xe8\xab\x9e\x1c\xc6\xc3\xbed\x90\x92oL\x0b\xf0\xa4h]\x01)\x9e0'

In [34]:
signature

b'\x93|\xa0t\xd7l\xd8\xd9\x0cG\xba\x90u1\x0c:e\xbe\x15\x83=\xf4\xf1\xe7\x05n\xe7#\x7f#\xf2tK6~\x123\xdf\xa2}\x13\xfc!%\xa4\x8f\xeap\x8fL\xb6\xc0\x8a\xbd\xbfY8\xcdj\x15\xea\x93_\xe2'

Let's encode them.

In [35]:
from base64 import b64encode, b64decode

In [36]:
public_key = b64encode(vk.to_string())
sign = b64encode(signature)

print('public key: {} \nsignature:  {}'.format(public_key, sign))

public key: b'BvqECTOdh5fXAoIFZMwL3jHD43yr9GyF/s57E+Sd6cGH9fnG2qSpElbD4+irnhzGw75kkJJvTAvwpGhdASmeMA==' 
signature:  b'k3ygdNds2NkMR7qQdTEMOmW+FYM99PHnBW7nI38j8nRLNn4SM9+ifRP8ISWkj+pwj0y2wIq9v1k4zWoV6pNf4g=='


That looks a little better. Are there any other possibilities?

In [37]:
from binascii import hexlify, unhexlify

In [38]:
public_key = hexlify(vk.to_string())
sign = hexlify(signature)

print('public key: {} \nsignature:  {}'.format(public_key, sign))

public key: b'06fa8409339d8797d702820564cc0bde31c3e37cabf46c85fece7b13e49de9c187f5f9c6daa4a91256c3e3e8ab9e1cc6c3be6490926f4c0bf0a4685d01299e30' 
signature:  b'937ca074d76cd8d90c47ba9075310c3a65be15833df4f1e7056ee7237f23f2744b367e1233dfa27d13fc2125a48fea708f4cb6c08abdbf5938cd6a15ea935fe2'


It looks prettier but longer. I'll stick with the shorter version here. I always wondered why wallet addresses look so different for different coins. I guess the answer lies in encoding preferences. Onward to our signature field!

In [39]:
# Our new transaction data
@dataclass
class Transaction:
    recipient: bytes
    sender: bytes
    amount: float
    signature: bytes = b'' # New signature filed
    timestamp: float = field(default_factory=time)

In [40]:
def sign(public: bytes, private: bytes) -> bytes:
    pk = b64decode(private)
    signing_key = ecdsa.SigningKey.from_string(pk, curve=ecdsa.SECP256k1)
    signature = b64encode(signing_key.sign(public))
    return signature


def is_valid(transaction: Transaction) -> bool:
    pub_key = ecdsa.VerifyingKey.from_string(b64decode(transaction.sender))
    return pub_key.verify(b64decode(transaction.signature), transaction.sender)


class Person:
    public: bytes
    private: bytes

    def __init__(self):
        sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
        vk = sk.get_verifying_key()
        self.public = b64encode(vk.to_string())
        self.private = b64encode(sk.to_string())

    def __repr__(self):
        return str(self.public)

    def send_help(self, to:bytes, amount: float) -> Transaction:
        return Transaction(
            recipient=to,
            sender=self.public,
            signature=sign(self.public, self.private),
            amount=amount
        )

Now if Alice wants to send help to Bob, first she and Bob have to generate key pairs.

In [41]:
alice = Person()
bob = Person()
jon = Person()

In [42]:
alice.send_help(to=bob.public, amount=2.0)

Transaction(recipient=b'wLJrx3m7PaAAHBznkGkaTcvKIv1RPiEYfhJXTLA2Dkv9WaTisxzH9/qaz0D/MJFcdoQzAeRlTHlIZXXb0IkHbA==', sender=b'6fSY2PmB8tfTcPZiiMravTkCkmWjYyTLewXoTQIAqD1aTyDdZ/k9866HzaUhw4FBOoeMqaEteIjMYDnFYvBERA==', amount=2.0, signature=b'LBFQyrIH6xq6d37q+rWxk9Msmwa+q0inZI+7eVvDwhLAJxQsiAdbENCdjWFPz4FJzqrB6cGnX3/EljlIXq+Nlg==', timestamp=1543081709.6904411)

In [43]:
chain = Chain()

In [44]:
block = Block()

In [46]:
block.append(alice.send_help(to=bob.public, amount=10.0))
block.append(bob.send_help(to=jon.public, amount=5.0))
block.append(jon.send_help(to=alice.public, amount=2.5))

In [47]:
add_block_with_proof_of_work(chain, block)

Found hash 00e84f8cca8b7ab7c0809863df7ac87339bfe0c92943e117b710abefa9cba645 after 433 iterations


In [48]:
chain.blocks

[transactions: [Transaction(recipient='Alice', sender='Alice', amount=2.0, signature=b'', timestamp=1543081711.6849842)]
 
 previous hash: coinbase
 
 nonce: 3367901e-0799-48d1-b46f-637e172e25c7,
 transactions: [Transaction(recipient=b'wLJrx3m7PaAAHBznkGkaTcvKIv1RPiEYfhJXTLA2Dkv9WaTisxzH9/qaz0D/MJFcdoQzAeRlTHlIZXXb0IkHbA==', sender=b'6fSY2PmB8tfTcPZiiMravTkCkmWjYyTLewXoTQIAqD1aTyDdZ/k9866HzaUhw4FBOoeMqaEteIjMYDnFYvBERA==', amount=10.0, signature=b'0iX4aF6tuNd5micYLE9d7W5gViBFeGsDzxuS8ExtfxPwnuwS+lxmuFjg2Z0Z8hh8Xopagwjn+O8GoRgkR1TLMw==', timestamp=1543081723.5875883), Transaction(recipient=b'PJSLBxxGFVOhu4n1nyoNOZGcC7lUdqpKIGVkOtkuJwjI6f2LwF3C7/Ff9RX3NNYs1X7X79N6JWbASwBcqxMocg==', sender=b'wLJrx3m7PaAAHBznkGkaTcvKIv1RPiEYfhJXTLA2Dkv9WaTisxzH9/qaz0D/MJFcdoQzAeRlTHlIZXXb0IkHbA==', amount=5.0, signature=b'MVcrroA8VRwhpsqlza1tdKfohzihHclBdgRycu0Q89Wz8dKf+8Pxs4IRPcAg33Rq2lprGM3OqmreLeG4LlCRPg==', timestamp=1543081723.6789649), Transaction(recipient=b'6fSY2PmB8tfTcPZiiMravTkCkmWjYyTLewXoTQIAq

In [49]:
block = Block()

In [50]:
block.append(alice.send_help(to=bob.public, amount=2.5))
block.append(bob.send_help(to=jon.public, amount=5.0))
block.append(jon.send_help(to=alice.public, amount=10.0))

In [51]:
add_block_with_proof_of_work(chain, block)

Found hash 00f9e4e0e6fa03900af95587639152f4370e06bd035ac3ee8056d75a9ce3e325 after 496 iterations


In [52]:
chain.blocks

[transactions: [Transaction(recipient='Alice', sender='Alice', amount=2.0, signature=b'', timestamp=1543081711.6849842)]
 
 previous hash: coinbase
 
 nonce: 3367901e-0799-48d1-b46f-637e172e25c7,
 transactions: [Transaction(recipient=b'wLJrx3m7PaAAHBznkGkaTcvKIv1RPiEYfhJXTLA2Dkv9WaTisxzH9/qaz0D/MJFcdoQzAeRlTHlIZXXb0IkHbA==', sender=b'6fSY2PmB8tfTcPZiiMravTkCkmWjYyTLewXoTQIAqD1aTyDdZ/k9866HzaUhw4FBOoeMqaEteIjMYDnFYvBERA==', amount=10.0, signature=b'0iX4aF6tuNd5micYLE9d7W5gViBFeGsDzxuS8ExtfxPwnuwS+lxmuFjg2Z0Z8hh8Xopagwjn+O8GoRgkR1TLMw==', timestamp=1543081723.5875883), Transaction(recipient=b'PJSLBxxGFVOhu4n1nyoNOZGcC7lUdqpKIGVkOtkuJwjI6f2LwF3C7/Ff9RX3NNYs1X7X79N6JWbASwBcqxMocg==', sender=b'wLJrx3m7PaAAHBznkGkaTcvKIv1RPiEYfhJXTLA2Dkv9WaTisxzH9/qaz0D/MJFcdoQzAeRlTHlIZXXb0IkHbA==', amount=5.0, signature=b'MVcrroA8VRwhpsqlza1tdKfohzihHclBdgRycu0Q89Wz8dKf+8Pxs4IRPcAg33Rq2lprGM3OqmreLeG4LlCRPg==', timestamp=1543081723.6789649), Transaction(recipient=b'6fSY2PmB8tfTcPZiiMravTkCkmWjYyTLewXoTQIAq

Okay, we sufficiently ridiculous-ified. However, it looks so scary only because of public keys and cryptographic signature. If we ignore those strings, things are actually very simple: 

1. There are transactions with who gave help to whom and a signature so there is a way to validate the transaction. 
2. There is a `nonce` field that we use to calculate hash for next block. 
3. There is a prevous hash field.

You probably noticed that anyone can send any amount of help units. How do we make sure that help units appear only from mining? We don't. We just make a rule that people can spend only as much as they have and only miners can assign value to themselves. As long as miners verify work of others, they will follow any kind of verifiable rules. In order to make sure a person doesn't spend more than she has, miners just need to sum up all the transactions involving that public address. A rule that transactions are valid only in rainy weather will not work because our chain doesn't contain this information.

## Is this all there is to blockchain and cryptocoins?

Yes, it covers main concepts. And no, I wouldn't trust more than a dollar to this particular one (even if we don't take into account lots of small missing parts). When I started this quest, I suspected that it's possible to uncover details that I wouldn't find just by reading articles about blockchains. The digest of what I learned while writing this simple blockchain is in `5 Things I learned from Implementing a Simple Blockchain`.

# 5 Things I learned from Implementing a Simple Blockchain

I went on a quest to implement cryptocurency concepts in code. In my prevous article `In code we trust`, I argued that one must impement cryptocurrency concepts in code in order to understand them. I still stand by that statement but now I understand why very few bloggers do that. It is very boring to read. Don't get me wrong; it was fun to think about it and write code but I must be honest with myself and admit that very few people will want to read it carefully. My `Chain of thought` jupyter notebook is for those brave souls. The rest of the article focuses on lessons I learned while writing code for a simple blockchain.

[pic]

## Not money
It's better to __not__ think of cryptocurrency as money. It's an immutable list of transactions. When you add entities that maintain the ledger and validate each other entries, the most apparant application is to "send" units of value from one address to another. The true value of such ledgers comes from enabling cooperation between people, autobeings, aliens, unicorns, or anything else that follows the protocol. 

## Not complicated
It turns out that the main concepts are very simple. I reconstructed them with very few lines of code starting from a simple list of transactions and making any attempts to tamper apparent. Specific implementation matters a lot. That is what separates successful coin worth millions of dollars from worthless crap.

## Not a database
I kind of knew that before but I didn't really appreciate how special the first and the last blocks are. Blockchain itself is a crappy database. It's only good for verifying chain integrity. It's incredibly inefficient even at that task. Any blockchain should have a better data format on top of it. It would be interesting to check what data formats are used by famous coins.

## Game of rules
It's fascinating that a lot of coin rules don't need to be spelled out explicitly in the blockchain. They are reinforced only by miner implementation. For example, blockchain doesn't need to contain wallet balances. It's calculated by summing up all the transactions with the wallet's public key. This is a cool feature but it's easy to create ambigious rules or rules with unintended consequences. Let's say we allow a negative balance in our transactions. It might work for a while until some smart soul realizes that she can create wallets, empty them down to that negative amount into another wallet, and abandon the ones with negative amounts. Even though the example is simple, the relationship between negative account balance and the ability to limitlessly print the coin is not very apparent. It seems to me that complexity increases with the number of those rules. With complexity the chance of unintended consequences increases as well.

## Validation right
Destributing validation right using proof of work is genius but it feels wrong. It reminds me the famous Arthur C. Clarke story [The Nine Billion Names of God](https://en.wikipedia.org/wiki/The_Nine_Billion_Names_of_God). No wonder people are looking for alternatives. I wish there was some law of quantum randomness that could distribute validation rights and didn't accept bribes. I doubt it's possible in this universe though.

## What's next?
First, I'm going to look into some real blockchain formats like Bitcoin, Ethereum, Steem, and maybe another interesting coin. There must be immutable database-like structures that are already implemented. After that, I'll work on some tools that can tell me some stats about blockchains. The rule is that the longest chain wins but I'm curious how many blocks die on average, replaced by a longer chain.

I'm going to continue to explore the cryptoverse through code. Any suggestions on which areas I should focus? 